In [3]:
import autogluon as ag
# from autogluon import TabularPrediction as task
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd ## read/write data
import re ## regex search for features name
from sklearn import preprocessing ## standardization
import math ## take log10
import numpy as np ## data handling
from autogluon.tabular import TabularDataset, TabularPredictor

NameError: name 'np' is not defined

In [4]:
label = 'Measured logKpf or logKmw'

In [5]:
def my_function(dir_v,pp,    fp,    refFeatures,    auto_stack,    hyperparameter_tune,    eval_metric,    num_bag_folds, load_model = False, outputPred = False, output_file = ''):
    output_path = dir_v + output_file + '.csv'
    # load train dat
    dataPath = dir_v + "dat_training.csv"
    train_data = TabularDataset(dataPath)
    fpCols = list(filter(re.compile("^V[0-9]+$").match, list(train_data.columns)))
    drop_ref_features = list(set(['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C','Temp (C)']) - set(refFeatures))
    if dir_v == "./logkpf_33Rank/":
        train_data = train_data.drop(columns=["CAS"])
    else:
        train_data = train_data.drop(columns=["CASfill0"])
    ppFeatures = list(set(list(train_data.columns)) - set(fpCols) - set(['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C','Temp (C)']) - set([label]))
    for fp_v in fpCols:
        train_data[fp_v] = train_data[fp_v].apply(str)
    if not fp:
        train_data = train_data.drop(columns=fpCols)
    if not pp:
        train_data = train_data.drop(columns=ppFeatures)
    if len(refFeatures) != 4:
        train_data = train_data.drop(columns=drop_ref_features)
    # load test dat
    test_data = TabularDataset(dir_v + "dat_test.csv")
    for fp_v in list(filter(re.compile("^V[0-9]+$").match, list(test_data.columns))):
        test_data[fp_v] = test_data[fp_v].apply(str)
    y_test = test_data[label]  # values to predict
    test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
    # set model save_path
    save_path = dir_v + 'agModels-predictClass'  # specifies folder to store trained models
    save_path = dir_v + 'agModels-predictClass' + '-' + str(refFeatures)
    if fp:
        save_path = save_path + '-fp'
    if pp:
        save_path = save_path + '-pp'
    if auto_stack:
        save_path = save_path + '-stack'
    if hyperparameter_tune:
        save_path = save_path + '-hyperparameter_tune'
    if load_model:
        predictor = TabularPredictor.load(save_path)
    else:        
        # training
        predictor = TabularPredictor(label=label, path=save_path, eval_metric = eval_metric).fit(train_data, num_bag_folds=num_bag_folds,excluded_model_types=['NN'],
                                                                                                           auto_stack=auto_stack, ag_args_fit={'num_gpus': 1})
        # write training res to file
        pd.DataFrame([[dir_v,str(refFeatures),padel,fp,auto_stack,hyperparameter_tune,eval_metric,str(num_bag_folds)]],
                     columns=['dir_v','ref','padel','fp','auto_stack','hyperparameter_tune','eval_metric','num_bag_folds']).to_csv('./autoGluon2.csv',mode='a')
        train_res = predictor.leaderboard()
        train_res["parInfo"] = str((dir_v,str(refFeatures),padel,fp,auto_stack,hyperparameter_tune,eval_metric,str(num_bag_folds)))
        train_res.to_csv(output_path,mode='a')
    # write predict res to file
    if outputPred:
        print(str((dir_v,str(refFeatures))))
        train_data_nolab = train_data.drop(columns=[label])
        y_train = train_data[label]  # values to predict
        p_model_train = predictor.predict(train_data_nolab,model='WeightedEnsemble_L2')
        perf = predictor.evaluate_predictions(y_true=y_train, y_pred = p_model_train, auxiliary_metrics=True)
        
        p_model_test = predictor.predict(test_data_nolab,model='WeightedEnsemble_L2')
        df = pd.DataFrame(p_model_train)
        df["classify"] = "training"
        df_test = pd.DataFrame(p_model_test)
        df_test["classify"] = "test"
        df_bag_folds = pd.DataFrame(predictor.get_oof_pred(model= 'WeightedEnsemble_L2', transformed=False, train_data=train_data, internal_oof=False))
        df_bag_folds["classify"] = "5_bag_folds"
        df = df.append(df_test)
        df = df.append(df_bag_folds)
        df["parInfo"] = str((dir_v,str(refFeatures)))
        df.to_csv(output_path,mode='a')
    else:
        m_name_list = predictor.get_model_names()
        p_model=predictor.predict(test_data_nolab,model=m_name_list[0])
        perf = predictor.evaluate_predictions(y_true=y_test, y_pred=p_model, auxiliary_metrics=True)
        predict_df = pd.DataFrame([list(perf.values())], columns=perf.keys())
        predict_df['model'] = m_name_list[0]
        predict_df["parInfo"] = str((dir_v,str(refFeatures),padel,fp,auto_stack,hyperparameter_tune,eval_metric,str(num_bag_folds)))
        del m_name_list[0]
        if len(m_name_list) <=1:
            m_name_list = [m_name_list]
        for m_name in m_name_list:
            print(m_name)
            p_model=predictor.predict(test_data_nolab,model=m_name)
            perf = predictor.evaluate_predictions(y_true=y_test, y_pred=p_model, auxiliary_metrics=True)
            df = pd.DataFrame([list(perf.values())], columns=perf.keys())
            df['model'] = m_name
            df["parInfo"] = str((dir_v,str(refFeatures),padel,fp,auto_stack,hyperparameter_tune,eval_metric,str(num_bag_folds)))
            predict_df = predict_df.append(df)
        predict_df.to_csv(output_path,mode='a')

In [6]:
#dir_v_list = ["./logkpf_33Rank/","./logkpf_splitByStudy/","./logkpf_splitByRandomPercentage/","./logkpf_splitByorderedResponse/","./logkpf_splitByOrderedStructure/"]
# dir_v_list = ["./logkpf_splitByStudy/","./logkpf_splitByRandomPercentage/","./logkpf_splitByorderedResponse/","./logkpf_splitByOrderedStructure/"]
dir_v_list = ["./logkpf_splitByStudy/"]
parameter_grid = [(dir_v,padel,fp,refFeatures,auto_stack,hyperparameter_tune) for dir_v in dir_v_list for padel in [False] \
                for fp in [True] for refFeatures in [['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C','Temp (C)'],
                                                          ['EtOH-eq (%)(v/v)']] \
                for auto_stack in [False] for hyperparameter_tune in [False]]
eval_metric = 'root_mean_squared_error'
num_bag_folds = 5
print(parameter_grid)
for par_items in parameter_grid:
    # dat/features/model_train parameter 
    print(par_items)
    dir_v = par_items[0]
    padel = par_items[1]
    fp = par_items[2]
    refFeatures = par_items[3]
    auto_stack = par_items[4]
    hyperparameter_tune = par_items[5]
    my_function(dir_v,padel,    fp,    refFeatures,    auto_stack,    hyperparameter_tune,    eval_metric,    num_bag_folds)

[('./logkpf_splitByStudy/', False, True, ['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C', 'Temp (C)'], False, False), ('./logkpf_splitByStudy/', False, True, ['EtOH-eq (%)(v/v)'], False, False)]
('./logkpf_splitByStudy/', False, True, ['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C', 'Temp (C)'], False, False)


Beginning AutoGluon training ...
AutoGluon will save models to "./logkpf_splitByStudy/agModels-predictClass-['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C', 'Temp (C)']-fp/"
AutoGluon Version:  0.1.0
Train Data Rows:    1481
Train Data Columns: 885
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (7.724242509, -3.508638306, 2.16598, 2.79632)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    328692.2 MB
	Train Data (Original)  Memory Usage: 75.81 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to

[1000]	train_set's rmse: 0.148675	valid_set's rmse: 0.308354


	Training S1F2 with GPU, note that this may negatively impact model quality compared to CPU training.
	Training S1F3 with GPU, note that this may negatively impact model quality compared to CPU training.


[1000]	train_set's rmse: 0.142317	valid_set's rmse: 0.344604
[2000]	train_set's rmse: 0.1258	valid_set's rmse: 0.340481


...

[1000]	train_set's rmse: 0.142317	valid_set's rmse: 0.34447
[2000]	train_set's rmse: 0.1258	valid_set's rmse: 0.340368


	Training S1F4 with GPU, note that this may negatively impact model quality compared to CPU training.


[1000]	train_set's rmse: 0.144201	valid_set's rmse: 0.354559
[2000]	train_set's rmse: 0.12328	valid_set's rmse: 0.347704


	Training S1F5 with GPU, note that this may negatively impact model quality compared to CPU training.
	-0.3338	 = Validation root_mean_squared_error score
	40.59s	 = Training runtime
	1.44s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	Training S1F1 with GPU, note that this may negatively impact model quality compared to CPU training.
	Training S1F2 with GPU, note that this may negatively impact model quality compared to CPU training.
	Training S1F3 with GPU, note that this may negatively impact model quality compared to CPU training.
	Training S1F4 with GPU, note that this may negatively impact model quality compared to CPU training.
	Training S1F5 with GPU, note that this may negatively impact model quality compared to CPU training.


KeyboardInterrupt: 

In [6]:
#dir_v_list = ["./logkpf_33Rank/","./logkpf_splitByStudy","./logkpf_splitByRandomPercentage/","./logkpf_splitByorderedResponse/","./logkpf_splitByOrderedStructure/"]
dir_v_list = ["./logkpf_splitByStudy/","./logkpf_splitByRandomPercentage/","./logkpf_splitByorderedResponse/","./logkpf_splitByOrderedStructure/"]
parameter_grid = [(dir_v,padel,fp,refFeatures,auto_stack,hyperparameter_tune) for dir_v in dir_v_list for padel in [False,True] \
                for fp in [True,False] for refFeatures in [['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C','Temp (C)'],
                                                          ['EtOH-eq (%)(v/v)', 'Material_consolidated', 'Temp (C)'],
                                                          ['EtOH-eq (%)(v/v)','Temp (C)'],
                                                          ['EtOH-eq (%)(v/v)']] \
                for auto_stack in [True,False] for hyperparameter_tune in [False]]
eval_metric = 'root_mean_squared_error'
num_bag_folds = 5
par_items = parameter_grid[0]
dir_v = par_items[0]
padel = par_items[1]
fp = par_items[2]
refFeatures = par_items[3]
auto_stack = par_items[4]
hyperparameter_tune = par_items[5]
my_function(dir_v,padel,    fp,    refFeatures,    auto_stack,    hyperparameter_tune,    eval_metric,    num_bag_folds, load_model = False)

Beginning AutoGluon training ...
AutoGluon will save models to "./logkpf_splitByStudy/agModels-predictClass-['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C', 'Temp (C)']-fp-stack/"
AutoGluon Version:  0.1.0
Train Data Rows:    1481
Train Data Columns: 885
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (7.724242509, -3.508638306, 2.16598, 2.79632)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    333911.45 MB
	Train Data (Original)  Memory Usage: 75.81 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadat

[1000]	train_set's rmse: 0.148675	valid_set's rmse: 0.308436


	Training S1F2 with GPU, note that this may negatively impact model quality compared to CPU training.
	Training S1F3 with GPU, note that this may negatively impact model quality compared to CPU training.


[1000]	train_set's rmse: 0.142317	valid_set's rmse: 0.344878
[2000]	train_set's rmse: 0.1258	valid_set's rmse: 0.340734


	Training S1F4 with GPU, note that this may negatively impact model quality compared to CPU training.


[1000]	train_set's rmse: 0.144201	valid_set's rmse: 0.354817
[2000]	train_set's rmse: 0.123421	valid_set's rmse: 0.349278


	Training S1F5 with GPU, note that this may negatively impact model quality compared to CPU training.
	-0.3346	 = Validation root_mean_squared_error score
	30.83s	 = Training runtime
	1.44s	 = Validation runtime
Fitting model: LightGBMXT_BAG_L1 ...
	Training S1F1 with GPU, note that this may negatively impact model quality compared to CPU training.


KeyboardInterrupt: 

In [3]:
#dir_v_list = ["./logkpf_33Rank/","./logkpf_splitByStudy","./logkpf_splitByRandomPercentage/","./logkpf_splitByorderedResponse/","./logkpf_splitByOrderedStructure/"]
dir_v_list = ["./logkpf_splitByStudy/"]
parameter_grid = [(dir_v,padel,fp,refFeatures,auto_stack,hyperparameter_tune) for dir_v in dir_v_list for padel in [False] \
                for fp in [True] for refFeatures in [['EtOH-eq (%)(v/v)']] \
                for auto_stack in [False] for hyperparameter_tune in [False]]
eval_metric = 'root_mean_squared_error'
num_bag_folds = 5
par_items = parameter_grid[0]
dir_v = par_items[0]
padel = par_items[1]
fp = par_items[2]
refFeatures = par_items[3]
auto_stack = par_items[4]
hyperparameter_tune = par_items[5]
my_function(dir_v,padel,    fp,    refFeatures,    auto_stack,    hyperparameter_tune,    eval_metric,    num_bag_folds, load_model = False)

NameError: name 'my_function' is not defined

In [ ]:
dir_v_list = ["./logkpf_splitByStudy/"]
parameter_grid = [(dir_v,padel,fp,refFeatures,auto_stack,hyperparameter_tune) for dir_v in dir_v_list for padel in [False] \
                for fp in [True] for refFeatures in [['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C','Temp (C)'],
                                                          ['EtOH-eq (%)(v/v)']] \
                for auto_stack in [False] for hyperparameter_tune in [False]]
eval_metric = 'root_mean_squared_error'
num_bag_folds = 5
par_items = parameter_grid[1]
dir_v = par_items[0]
padel = par_items[1]
fp = par_items[2]
refFeatures = par_items[3]
auto_stack = par_items[4]
hyperparameter_tune = par_items[5]
my_function(dir_v,padel,    fp,    refFeatures,    auto_stack,    hyperparameter_tune,    eval_metric,    num_bag_folds, load_model = True)

In [ ]:
## AD

In [ ]:
dir_v_list = ["./logkpf_splitByStudy/","./logkpf_splitByRandomPercentage/","./logkpf_splitByorderedResponse/","./logkpf_splitByOrderedStructure/"]
parameter_grid = [(dir_v,padel,fp,refFeatures,auto_stack,hyperparameter_tune) for dir_v in dir_v_list for padel in [False] \
                for fp in [True] for refFeatures in [['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C','Temp (C)'],
                                                          ['EtOH-eq (%)(v/v)', 'Material_consolidated', 'Temp (C)'],
                                                          ['EtOH-eq (%)(v/v)','Temp (C)'],
                                                          ['EtOH-eq (%)(v/v)']] \
                for auto_stack in [True] for hyperparameter_tune in [False]]
eval_metric = 'root_mean_squared_error'
num_bag_folds = 5
for par_items in parameter_grid:
    dir_v = par_items[0]
    padel = par_items[1]
    fp = par_items[2]
    refFeatures = par_items[3]
    auto_stack = par_items[4]
    hyperparameter_tune = par_items[5]
    my_function(dir_v,padel,    fp,    refFeatures,    auto_stack,    hyperparameter_tune,    eval_metric,    num_bag_folds, load_model = False, outputPred = True, 
                output_file = 'autoGluon-predict-EM-cv2.csv')

In [ ]:
dir_v_list = ["./logkpf_splitByStudy/","./logkpf_splitByRandomPercentage/","./logkpf_splitByorderedResponse/","./logkpf_splitByOrderedStructure/"]
parameter_grid = [(dir_v,padel,fp,refFeatures,auto_stack,hyperparameter_tune) for dir_v in dir_v_list for padel in [False] \
                for fp in [True] for refFeatures in [['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C','Temp (C)'],
                                                          ['EtOH-eq (%)(v/v)', 'Material_consolidated', 'Temp (C)'],
                                                          ['EtOH-eq (%)(v/v)','Temp (C)'],
                                                          ['EtOH-eq (%)(v/v)']] \
                for auto_stack in [False] for hyperparameter_tune in [False]]
eval_metric = 'root_mean_squared_error'
num_bag_folds = 5
par_items = parameter_grid[0]
dir_v = par_items[0]
padel = par_items[1]
fp = par_items[2]
refFeatures = par_items[3]
auto_stack = par_items[4]
hyperparameter_tune = par_items[5]
my_function(dir_v,padel,    fp,    refFeatures,    auto_stack,    hyperparameter_tune,    eval_metric,    num_bag_folds, load_model = True, outputPred = True, 
            output_file = 'autoGluon-cv.csv')

In [16]:
dir_v_list = ["./logkpf_splitByStudy/","./logkpf_splitByRandomPercentage/","./logkpf_splitByorderedResponse/","./logkpf_splitByOrderedStructure/"]
parameter_grid = [(dir_v,padel,fp,refFeatures,auto_stack,hyperparameter_tune) for dir_v in dir_v_list for padel in [False] \
                for fp in [True] for refFeatures in [['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C','Temp (C)'],
                                                          ['EtOH-eq (%)(v/v)', 'Material_consolidated', 'Temp (C)'],
                                                          ['EtOH-eq (%)(v/v)','Temp (C)'],
                                                          ['EtOH-eq (%)(v/v)']] \
                for auto_stack in [False] for hyperparameter_tune in [False]]
eval_metric = 'root_mean_squared_error'
num_bag_folds = 5
par_items = parameter_grid[0]
dir_v = par_items[0]
padel = par_items[1]
fp = par_items[2]
refFeatures = par_items[3]
auto_stack = par_items[4]
hyperparameter_tune = par_items[5]
dataPath = dir_v + "dat_training.csv"
train_data = TabularDataset(dataPath)
fpCols = list(filter(re.compile("^V[0-9]+$").match, list(train_data.columns)))
drop_ref_features = list(set(['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C','Temp (C)']) - set(refFeatures))
if dir_v == "./logkpf_33Rank/":
    train_data = train_data.drop(columns=["CAS"])
else:
    train_data = train_data.drop(columns=["CASfill0"])
ppFeatures = list(set(list(train_data.columns)) - set(fpCols) - set(['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C','Temp (C)']) - set([label]))
for fp_v in fpCols:
    train_data[fp_v] = train_data[fp_v].apply(str)
if not fp:
    train_data = train_data.drop(columns=fpCols)
if not padel:
    train_data = train_data.drop(columns=ppFeatures)
if len(refFeatures) != 4:
    train_data = train_data.drop(columns=drop_ref_features)
save_path = dir_v + 'agModels-predictClass' + '-' + str(refFeatures)
output_file = 'autoGluon-cv.csv'
output_path = dir_v + output_file + '.csv'
print(save_path)
predictor = TabularPredictor.load(save_path)
predictor.leaderboard()
# print(predictor.get_oof_pred(model= 'WeightedEnsemble_L2', transformed=False, train_data=train_data, internal_oof=False))
# df = predictor.get_oof_pred(model= 'WeightedEnsemble_L2', transformed=False, train_data=train_data, internal_oof=False)
# df.to_csv(output_path,mode='a')

./logkpf_splitByStudy/agModels-predictClass-['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C', 'Temp (C)']
                    model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0     WeightedEnsemble_L2  -0.376952       0.869334  114.705660                0.000521           0.311429            2       True          9
1         LightGBM_BAG_L1  -0.390608       0.057468   17.404217                0.057468          17.404217            1       True          5
2       LightGBMXT_BAG_L1  -0.390608       0.059968   17.308017                0.059968          17.308017            1       True          6
3         CatBoost_BAG_L1  -0.397550       0.058614   75.643989                0.058614          75.643989            1       True          7
4    LightGBMLarge_BAG_L1  -0.421293       0.042996   25.971524                0.042996          25.971524            1       True          8
5  RandomForestMSE_BAG_L1  -0.42

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.376952,0.869334,114.705660,0.000521,0.311429,2,True,9
1,LightGBM_BAG_L1,-0.390608,0.057468,17.404217,0.057468,17.404217,1,True,5
2,LightGBMXT_BAG_L1,-0.390608,0.059968,17.308017,0.059968,17.308017,1,True,6
3,CatBoost_BAG_L1,-0.397550,0.058614,75.643989,0.058614,75.643989,1,True,7
4,LightGBMLarge_BAG_L1,-0.421293,0.042996,25.971524,0.042996,25.971524,1,True,8
5,RandomForestMSE_BAG_L1,-0.425396,0.707265,6.324027,0.707265,6.324027,1,True,1
6,ExtraTreesMSE_BAG_L1,-0.433761,0.692762,4.038008,0.692762,4.038008,1,True,2
7,KNeighborsDist_BAG_L1,-0.560413,0.187832,0.038001,0.187832,0.038001,1,True,4
8,KNeighborsUnif_BAG_L1,-0.626383,0.191761,0.038090,0.191761,0.038090,1,True,3


In [6]:
def buildModel(dir_v,pp,    fp,    refFeatures,    auto_stack,    hyperparameter_tune,    eval_metric,    num_bag_folds, load_model = False, outputPred = False, output_file = ''):
    output_path = dir_v + output_file + '.csv'
    # load train dat
    dataPath = dir_v + "dat_training.csv"
    train_data = TabularDataset(dataPath)
    fpCols = list(filter(re.compile("^V[0-9]+$").match, list(train_data.columns)))
    drop_ref_features = list(set(['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C','Temp (C)']) - set(refFeatures))
    if dir_v == "./logkpf_33Rank/":
        train_data = train_data.drop(columns=["CAS"])
    else:
        train_data = train_data.drop(columns=["CASfill0"])
    ppFeatures = list(set(list(train_data.columns)) - set(fpCols) - set(['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C','Temp (C)']) - set([label]))
    for fp_v in fpCols:
        train_data[fp_v] = train_data[fp_v].apply(str)
    if not fp:
        train_data = train_data.drop(columns=fpCols)
    if not pp:
        train_data = train_data.drop(columns=ppFeatures)
    if len(refFeatures) != 4:
        train_data = train_data.drop(columns=drop_ref_features)
    save_path = dir_v + 'agModels-predictClass' + '-' + str(refFeatures)
    if fp:
        save_path = save_path + '-fp'
    if pp:
        save_path = save_path + '-pp'
    if auto_stack:
        save_path = save_path + '-stack'
    if hyperparameter_tune:
        save_path = save_path + '-hyperparameter_tune'
    if load_model:
        predictor = TabularPredictor.load(save_path)
    else:        
        # training
        predictor = TabularPredictor(label=label, path=save_path, eval_metric = eval_metric).fit(train_data, num_bag_folds=num_bag_folds,excluded_model_types=['NN'],
                                                                                                           auto_stack=auto_stack)
        # write training res to file
        pd.DataFrame([[dir_v,str(refFeatures),padel,fp,auto_stack,hyperparameter_tune,eval_metric,str(num_bag_folds)]],
                     columns=['dir_v','ref','padel','fp','auto_stack','hyperparameter_tune','eval_metric','num_bag_folds']).to_csv('./autoGluon2.csv',mode='a')
        train_res = predictor.leaderboard()
        train_res["parInfo"] = str((dir_v,str(refFeatures),padel,fp,auto_stack,hyperparameter_tune,eval_metric,str(num_bag_folds)))
        train_res.to_csv(output_path,mode='a')
    # write predict res to file
    if outputPred:
        print(str((dir_v,str(refFeatures))))
        train_data_nolab = train_data.drop(columns=[label])
        y_train = train_data[label]  # values to predict
        p_model_train = predictor.predict(train_data_nolab,model='WeightedEnsemble_L2')
        df_train = pd.DataFrame(p_model_train)
        df_train["classify"] = "training"
        df_train["parInfo"] = str((dir_v,str(refFeatures)))        
        df = pd.DataFrame(predictor.get_oof_pred(model= 'WeightedEnsemble_L2', transformed=False, train_data=train_data, internal_oof=False))
        df["classify"] = "5_bag_folds"
        df["parInfo"] = str((dir_v,str(refFeatures)))
        df = df.append(df_train)
        df.to_csv(output_path,mode='a')
    else:
        m_name_list = predictor.get_model_names()

In [7]:
dir_v_list = ["./notSplit/"]
parameter_grid = [(dir_v,padel,fp,refFeatures,auto_stack,hyperparameter_tune) for dir_v in dir_v_list for padel in [False] \
                for fp in [True] for refFeatures in [ ['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C','Temp (C)'],
                                                      ['EtOH-eq (%)(v/v)', 'Material_consolidated', 'Temp (C)'],
                                                      ['EtOH-eq (%)(v/v)','Temp (C)'],
                                                      ['EtOH-eq (%)(v/v)']] \
                for auto_stack in [False] for hyperparameter_tune in [False]]
eval_metric = 'root_mean_squared_error'
num_bag_folds = 5
print(parameter_grid)
for par_items in parameter_grid:
    # dat/features/model_train parameter 
    print(par_items)
    dir_v = par_items[0]
    padel = par_items[1]
    fp = par_items[2]
    refFeatures = par_items[3]
    auto_stack = par_items[4]
    hyperparameter_tune = par_items[5]
    buildModel(dir_v,padel,    fp,    refFeatures,    auto_stack,    hyperparameter_tune,    eval_metric,    num_bag_folds, load_model = True, outputPred = True, 
                output_file = 'autoGluon-predict-EM-cv-training.csv')

[('./notSplit/', False, True, ['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C', 'Temp (C)'], False, False), ('./notSplit/', False, True, ['EtOH-eq (%)(v/v)', 'Material_consolidated', 'Temp (C)'], False, False), ('./notSplit/', False, True, ['EtOH-eq (%)(v/v)', 'Temp (C)'], False, False), ('./notSplit/', False, True, ['EtOH-eq (%)(v/v)'], False, False)]
('./notSplit/', False, True, ['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C', 'Temp (C)'], False, False)
('./notSplit/', "['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C', 'Temp (C)']")
('./notSplit/', False, True, ['EtOH-eq (%)(v/v)', 'Material_consolidated', 'Temp (C)'], False, False)
('./notSplit/', "['EtOH-eq (%)(v/v)', 'Material_consolidated', 'Temp (C)']")
('./notSplit/', False, True, ['EtOH-eq (%)(v/v)', 'Temp (C)'], False, False)
('./notSplit/', "['EtOH-eq (%)(v/v)', 'Temp (C)']")
('./notSplit/', False, True, ['EtOH-eq (%)(v/v)'], False, False)
('./notSplit/', "['EtOH-eq (%)(v/v)']")
